## **Image classification**


จากบทเรียนที่น้องๆได้ลองเรียนมานั้น จะเห็นว่า Deep learning สามารถนำไปใช้งานได้หลากหลายวิธีมากๆ หนึ่งในงานที่ใช้กันแพร่หลายมากคือ Image classification

โดยในบทเรียนแรกนี้เราจะมาลองทำ Image classification กันโดยใช้ library ที่ชื่อว่า Pytorch และ Pytorch Lightning

โดย Dataset ที่เราจะเลือกใช้ในวันนี้จะใช้ชุดข้อมูลจาก Kaggle https://www.kaggle.com/c/dog-breed-identification

Reference:
- https://github.com/udacity/deep-learning-v2-pytorch
- https://albumentations.ai/docs/getting_started/image_augmentation/

In [ ]:
# ! นำหน้าใน Notebook แปลว่าให้รันใน Terminal
!pip install kaggle
!kaggle competitions download -c dog-breed-identification

In [ ]:
# ทำการ unzip .zip ไฟล์ที่ดาวน์โหลดมา
!unzip dog-breed-identification.zip -d data

In [ ]:
!pip install torchvision
!pip install pytorch-lightning

## **แบ่งชุดข้อมูลเป็น training, validation set**

In [ ]:
import os
import os.path as op
import shutil
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from torchvision import datasets, models, transforms
import torchvision.transforms as T
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
# get all image paths
img_df = pd.DataFrame(glob("data/train/*.jpg"), columns=["path"])
img_df["id"] = img_df.path.map(lambda x: op.basename(x).replace(".jpg", ""))

# read label data
label_df = pd.read_csv("data/labels.csv")
train_df = img_df.merge(label_df, on="id")

In [ ]:
train_df.head()

In [ ]:
train_df, validation_df = train_test_split(train_df, test_size=0.2, random_state=3)

In [ ]:
print("Length of training set = {}, validation set = {}".format(len(train_df), len(validation_df)))

In [ ]:
root_dir = "data/dogdata/"
for df, f in zip([train_df, validation_df], ["train", "validation"]):
    for _, r in df.iterrows():
        # create subfolder if it doesn't exist
        d = op.join(root_dir, f, r.breed)
        if not op.exists(d):
            os.makedirs(d)
        shutil.copy(r.path, op.join(root_dir, f, r.breed, f"{r.id}.jpg"))

## **Image classification ด้วย Pytorch ล้วนๆ**

In [ ]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import torch.nn.functional as F
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

In [ ]:
train_transform = T.Compose([
    T.Resize(256)
    T.RandomHorizontalFlip(p=0.5),
    T.TrivialAugmentWide(),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
val_transform = T.Compose([
    T.Resize(224),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),)
])

In [ ]:
train_data = datasets.ImageFolder("data/dogdata/train/", transform=train_transform)
val_data = datasets.ImageFolder("data/dogdata/validation/", transform=val_transform)

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [ ]:
n_train = len(train_loader.dataset)
n_val = len(val_loader.dataset)

In [ ]:
# ตัวอย่างการโหลดออกมา 1 batch
images, labels = next(iter(train_loader))

In [ ]:
# 16 = batch size, 3 = depth, 224 = height, 224 = width
images.shape, labels.shape

In [ ]:
idx2_class = {v: k for k, v in train_data.class_to_idx.items()}
fig = plt.figure(figsize=(25, 4))
for i in range(10):
    image = np.transpose(images.cpu()[i])
    label = idx2_class[labels.cpu().tolist()[i]]
    ax = fig.add_subplot(2, 8, i + 1, xticks=[], yticks=[])
    plt.imshow(image)
    ax.set_title(label)

In [ ]:
# สร้าง pretrained model
model = models.resnet34(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# เปลี่ยนพารามิเตอร์ใน fc ให้สำหรับทำนายพันธุ์หมาแทน, เช็คก่อนเสมอว่าโมเดลหน้าตาเป็นอย่างไร
model.fc = nn.Linear(in_features=512, out_features=len(train_data.classes))

In [ ]:
# สร้าง loss, optimizer
cross_entropy = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=2e-3)

In [ ]:
# เช็คว่ามี GPU ที่สามารถใช้ได้มั้ย ถ้าใช้ได้นำโมเดลเข้าไปอยู่ใน GPU
gpu = torch.cuda.is_available()
print(gpu)
if gpu:
    model.cuda()

In [ ]:
n_epochs = 30
for epoch in range(n_epochs):
    # ช่วง train
    model.train()
    train_loss, val_loss = 0, 0
    for images, labels in tqdm(train_loader):
        if gpu:
            images, labels = images.cuda(), labels.cuda()
        optimizer.zero_grad()
        pred = model(images) # คำนวณหา output (pred) จาก โมเดลที่มีอยู่
        loss = cross_entropy(pred, labels)
        loss.backward() # คำนวณ gradient จาก loss ที่ได้
        optimizer.step() # อัพเดทพารามิเตอร์ของโมเดล
        train_loss += loss.item() * images.size(0)

    # ช่วง validate
    model.eval() # เซ็ตเป็น evaluation mode
    for images, labels in tqdm(val_loader):
        if gpu:
            images, labels = images.cuda(), labels.cuda()
        pred = model(images)
        loss = cross_entropy(pred, labels)
        val_loss += loss.item() * images.size(0)
    print("Training loss = {}, Validation loss = {}".format(train_loss / n_train, val_loss / n_val))

In [ ]:
# คำนวณหา classification report สำหรับ validation set
y_pred, y_true = [], []
model.eval() # เซ็ตเป็น evaluation mode
for images, labels in tqdm(val_loader):
    if gpu:
        images, labels = images.cuda(), labels.cuda()
    pred = model(images)
    yp = pred.argmax(dim=1).tolist()
    yt = labels.tolist()
    y_pred.extend(yp)
    y_true.extend(yt)
print(classification_report(y_true, y_pred))

In [ ]:
print("Accuracy on validation set = {}".format(
    accuracy_score(y_true, y_pred))
)

In [ ]:
print("Precision Recall F1-Score",
      precision_recall_fscore_support(y_true, y_pred, average="micro"))

## **ทดลอง Forward pass**

เราสามารถลองดูได้ว่า forward pass ของเราทำงานได้ปกติมั้ย

In [ ]:
x, y = next(iter(train_loader))
backbone = models.resnet34(pretrained=True)
backbone.fc = torch.nn.Linear(backbone.fc.in_features, n_classes)

In [ ]:
y_hat = backbone(x)

In [ ]:
y_hat.shape, y.shape # batch size, number of classes

In [ ]:
entropy_loss = nn.CrossEntropyLoss()
entropy_loss(y_hat, y)

## **Image classification ด้วย Pytorch Lightning**

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torchmetrics import Accuracy
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
train_transform = T.Compose([
    T.Resize((256, 256)),
    T.RandomHorizontalFlip(p=0.5),
    T.TrivialAugmentWide(),
    T.RandomResizedCrop((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),)
])

In [ ]:
train_data = datasets.ImageFolder("data/dogdata/train/", transform=train_transform)
val_data = datasets.ImageFolder("data/dogdata/validation/", transform=val_transform)

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [ ]:
classes = train_data.classes
n_classes = len(classes)

In [ ]:
class DogResNet(pl.LightningModule):
    def __init__(self, n_classes=120):
        super(DogResNet, self).__init__()
        
        # จำนวนของพันธุ์น้องหมา (120)
        self.n_classes = n_classes

        # ใช้สถาปัตยกรรม resnet34; เปลี่ยน layer สุดท้าย
        self.backbone = models.resnet34(pretrained=True)
        for param in self.backbone.parameters():
            param.requires_grad = False
        # เปลี่ยน fc layer เป็น output ขนาด 120
        self.backbone.fc = torch.nn.Linear(self.backbone.fc.in_features, n_classes)
        
        self.entropy_loss = nn.CrossEntropyLoss()
        self.accuracy = Accuracy()

    def forward(self, x):
        preds = self.backbone(x)
        return pred

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.backbone(x)
        loss = self.entropy_loss(logits, y)
        y_pred = torch.argmax(logits, dim=1)
        self.log("train_loss", loss)
        self.log("train_acc", self.accuracy(y_pred, y))
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.backbone(x)
        loss = self.entropy_loss(logits, y)
        y_pred = torch.argmax(logits, dim=1)
        self.log("val_loss", loss)
        self.log("val_acc", self.accuracy(y_pred, y))
        return loss
        
    def configure_optimizers(self):
        self.optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        return {
            "optimizer": self.optimizer,
            "monitor": "val_loss",
        }

In [ ]:
model = DogResNet(n_classes=n_classes)

In [ ]:
checkpoint_callback = ModelCheckpoint(
   dirpath="./checkpoints/dogbreed/",
   filename="resnet18--{epoch:02d}-{val_acc:.2f}-{val_loss:.2f}",
   save_top_k=1,
   verbose=True,
   monitor="val_loss",
   mode="min",
)

In [ ]:
trainer = pl.Trainer(max_epochs=10, gpus=1, callbacks=[checkpoint_callback])
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)